<a href="https://colab.research.google.com/github/ShanMehsood/Car-Detection-and-Counting-using-YOLOv5/blob/main/3_final_board_detection_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import cv2
import glob

In [3]:
cfg_model_path='car_best.pt'

In [6]:
model=torch.hub.load('ultralytics/yolov5', 'custom', path=cfg_model_path, force_reload=True)
model.to('cpu')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-7-6 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)



Exception: [Errno 2] No such file or directory: 'car_best.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.

In [7]:
#reading video file and resizing into 380x640 size and passing it to model
car=0
cy1=320
offset=4.5
frame_counter=0
cap = cv2.VideoCapture('videocar.mp4')
while True:
    ret, frame = cap.read()
    frame_counter+=1
    if ret == False:
        break
    frame = cv2.resize(frame, (640, 380))
    frame_save=frame.copy()
    cv2.line(frame,(0,cy1),(640,cy1),(0,0,255),1)
    results = model(frame)
    a=results.pandas().xyxy[0]
    for index, row in a.iterrows():
        if row['class'] == 0:
            x1=int(row['xmin'])
            y1=int(row['ymin'])
            x2=int(row['xmax'])
            y2=int(row['ymax'])

            rectx1,recty1=((x1+x2)/2),((y1+y2)/2)
            rectcenter=int(rectx1),int(recty1)
            cx=rectcenter[0]
            cy=rectcenter[1]
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.circle(frame,(cx,cy), 5, (0, 0, 255), -1)
            if cy>cy1-offset and cy<cy1+offset:
                car+=1
                #creating csv file to save frame number and board count
                with open('car_coun.csv', 'a') as f:
                    f.write(str(car)+','+str(frame_counter)+'\n')
                #creating a folder to save the frame with board count
                cv2.imwrite('./car_img/'+str(car)+'.jpg',frame_save)



    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()